# Доверительные интервалы для среднего

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

In [22]:
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import _tconfint_generic, _zconfint_generic
from scipy import stats

In [23]:
data = pd.read_csv('datasets/water.txt', sep='\t')
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [24]:
mean = data['mortality'].mean()

In [25]:
std = data['mortality'].std(ddof=1)

In [26]:
n = len(data)

In [27]:
_tconfint_generic(mean, std/n**0.5, alpha=0.05, dof=n-1, alternative='two-sided')

(1476.0833413552848, 1572.2117406119285)

Постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [28]:
south_data = data[data.location == 'South']
south_data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
2,South,Birmingham,1466,5
7,South,Bournemouth,1299,78
9,South,Brighton,1359,84
10,South,Bristol,1392,73


In [29]:
_tconfint_generic(south_data.mortality.mean(), south_data.mortality.std(ddof=1)/np.sqrt(len(south_data)), 
                  len(south_data)-1, 0.05, 'two-sided')

(1320.1517462936238, 1433.463638321761)

In [30]:
north_data = data[data.location == 'North']
north_data.head()

,location,town,mortality,hardness
1,North,Birkenhead,1668,17
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18
5,North,Bolton,1558,10
6,North,Bootle,1807,15


Постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

In [31]:
_tconfint_generic(north_data.mortality.mean(), north_data.mortality.std(ddof=1)/np.sqrt(len(south_data)), 
                  len(south_data)-1, 0.05, 'two-sided')

(1578.2899845852787, 1688.910015414721)

**Интервалы не пересекаются; видимо, средняя смертность на севере и на юге существенно разная**

Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [32]:
_tconfint_generic(south_data.hardness.mean(), south_data.hardness.std(ddof=1)/np.sqrt(len(south_data)), 
                  len(south_data)-1, 0.05, 'two-sided')

(53.467198692036106, 86.07126284642544)

In [33]:
_tconfint_generic(north_data.hardness.mean(), north_data.hardness.std(ddof=1)/np.sqrt(len(south_data)), 
                  len(south_data)-1, 0.05, 'two-sided')

(19.84404952047452, 40.95595047952548)

**Ответ:** не пересекаются

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией $\sigma^2: \bar{X}_n\pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}} $ 

При $\sigma=1$ какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью $\pm0.1$?

In [34]:
std_mean = 1
n = 1
left, right = _zconfint_generic(mean, std_mean/np.sqrt(n), 0.05, 'two-sided')
while (right - left)/2 > 0.1:
    n += 1
    left, right = _zconfint_generic(mean, std_mean/np.sqrt(n), 0.05, 'two-sided')
n

385

In [35]:
#альтернативный вариант
np.ceil((stats.norm.ppf(1-0.05/2) / 0.1)**2)

385.0

**Выберите правильные утверждния.**

Объём выборки, необходимый для построения доверительного интервала заданной ширины:
    1. уменьшается с ростом α
    2. увеличивается с ростом α
    3. увеличивается с увеличением требуемой ширины
    4. уменьшается с ростом дисперсии выборки
    5. увеличивается с ростом дисперсии выборки
    6. увеличивается с уменьшением требуемой ширины

In [36]:
alphas = [0.05, 0.1, 0.15, 0.30]
for alpha in alphas:
    print(np.ceil((stats.norm.ppf(1-alpha/2) / 0.1)**2))

385.0
271.0
208.0
108.0


In [37]:
inc_widths = [0.1, 0.2, 0.3, 0.4]
for width in inc_widths:
    print(np.ceil((stats.norm.ppf(1-0.05/2) / width)**2))

385.0
97.0
43.0
25.0


**Ответ:** 1, 5, 6